# Deployment

Goal: Fill out this notebook to import your trained model and build a Gradio interface.

After mocking up Gradio interface: Deploy your model to HuggingFace Spaces

In [33]:
!pip install gradio

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pysoundfile


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2022.9.24          |   py37h06a4308_0         154 KB
    conda-22.9.0               |   py37h06a4308_0         878 KB
    openssl-1.1.1q             |       h7f8727e_0         2.5 MB
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2022.9.2~ --> pkgs/main::ca-certificates-2022.07.19-h06a4308_0 None
  certifi            fastchan/noarch::certifi-2022.9.24-py~ --> pkgs/main/linux-64::certifi-2022.9.24-py37h06a4308_0 None
  conda               fastchan::conda-22.9.0-py37h89c1867_0 --> pkgs/main::conda-22.9.0-p

openssl-1.1.1q       | 2.5 MB    | ##################################### | 100% 
conda-22.9.0         | 878 KB    | ##################################### | 100% 
certifi-2022.9.24    | 154 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: ...working... done


In [15]:
# This file contains all the main external libs we'll use
from fastai.imports import * #used for fastai
from IPython import display #used to display media in notebook
import matplotlib.pyplot as plt #used to plot in notebook

from fastai.data.all import *
from fastai.data.external import *
from fastai.vision.all import *

import librosa;
import librosa.display;

import PIL as Pillow;
import gradio as gr;
import soundfile as sf;
import tempfile;

#### Load the learner from `.pkl` file. 

This will complain that you need some functions that aren't available in the namespace. Copy the necessary ones from the previous notebook.

In [5]:
# TODO: put the functions that are missing from the namespace here
# ....

We have to define a prediction function for our model:

In [6]:
learn = load_learner('model.pkl')

In [31]:
labels = learn.dls.vocab

def predict(audio):
    # grab data from Gradio upload
    sample_rate, data = audio 
    
    # recall that our dataset loads paths to audio files first, not the files themselves
    # lets make a temporary (in-memory) file
    temp_file = tempfile.NamedTemporaryFile(suffix='.aiff')
    
    # use soundfile library to write to temp file
    sf.write(temp_file.name, data, sample_rate)
    
    # get our prediction results
    pred,pred_idx,probs = learn.predict(Path(temp_file.name))
    
    # close tempfile
    temp_file.close()
    
    # return prediction results
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

In [29]:
title = "North Atlantic Right Whale Classifier"
description = "A NARW up-call classifier trained on the The Marinexplore and Cornell University Whale Detection Challenge dataset (Kaggle) with fastai."
article="<p style='text-align: center'><a href='https://www.kaggle.com/competitions/whale-detection-challenge' target='_blank'>Dataset</a></p>"
enable_queue=True

gr.Interface(fn=predict, inputs=gr.Audio(type="numpy"), outputs=gr.outputs.Label(num_top_classes=2),title=title,description=description,article=article,allow_flagging="never").launch(share=True)

/opt/conda/lib/python3.7/site-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  "Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components",
/opt/conda/lib/python3.7/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7869
Running on public URL: https://20935.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7ff86c52d6d0>,
 'http://127.0.0.1:7869/',
 'https://20935.gradio.app')

#### Your Job

Deploy to HuggingFace Spaces